In [44]:
import pandas as pd
from selenium import webdriver 
driver = webdriver.Chrome("../chromedriver_90")

url = "https://www.melon.com/chart/index.htm"

driver.get(url)

In [45]:
import time
# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()
time.sleep(1)
# 월간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a').click()
time.sleep(1)
# 연대선택 2020년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 연도선택 2021년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 월선택 1월 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 장르선택 종합 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 검색버튼 클릭
driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

In [62]:
# 월별 음악차트 50개 음악 번호 추출
from bs4 import BeautifulSoup
import urllib.request
import re
# xpath를 살짝 바꿨습니다. 50위까지는 클래스가 lst 50이지만 이후부터는 클래스가 달라져서요.
a_all = driver.find_elements_by_xpath('//*/td[4]/div/a')

#코드 짤 때 편의를 위해 stopper를 넣었습니다.
stopper = 0
for i in a_all:
    # (1)a.href values
    num = i.get_attribute('href')
    # (1)에서 숫자만 빼오기
    song_num = re.findall(r'\d+', num)
    # list -> string
    song_num = ",".join(song_num)
    # URL 결합
    song_url = "https://www.melon.com/song/detail.htm?songId=" + str(song_num)
    # 나의 헤더 찾기 [참조](https://wikidocs.net/66821)
    header =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    sequens = urllib.request.Request(song_url, headers = header)
    page = urllib.request.urlopen(sequens)
    soup = BeautifulSoup(page, 'html.parser', from_encoding="CP949")
    song_info = soup.find_all('div', class_="wrap_info")
    
    #데이터를 집어넣을 리스트 만들기
    title_list = []
    artist_list = []
    album_list = []
    release_list = []
    genre_list = []
    flac_list = []
    like_list =[]
    review_list=[]
    ranking_yest_list = []
    lyric_list=[]
    
    for j in song_info:
        #곡명
        title = j.find("div", class_="song_name")
        title_list.append(title.get_text(strip=True).replace("곡명", ""))
        #가수 - try: 가수가 1명인 경우/except:가수가 여러명인 경우
        try:
            artist = j.find("div", class_="artist")
            artist_list.append(artist.get_text(strip=True))
        except:
            artist_group = j.find("div", class_="atist_view d_layer d_dropdown").find("ul")
            for artist in artist_group:
                artist_list.append(artist.get_text(strip=True))
        
        #매타 항목 가져오기
        meta = j.find("div", class_="meta").find("dl").find_all("dd")
        #앨범
        album = meta[0]
        album_list.append(name.get_text(strip=True))        
        #발매일
        release = meta[1]
        release_list.append(release.get_text(strip=True))
        #장르
        genre = meta[2]
        genre_list.append(genre.get_text(strip=True))
        #FLAC: 있는 곡도 있고 없는 곡도 있으므로 없을 때에는 "na"출력
        try:
            flac = meta[3]
            flac_list.append(flac.get_text(strip=True))
        except:
            flac_list.append("na")
        #좋아요
        like = soup.find("span",id = "d_like_count")
        like_list.append(like.get_text(strip=True))
        #댓글 갯수
        review = soup.find(id = "revCnt")
        review_list.append(review.get_text(strip=True))
        #어제의 차트 순위: 없을 경우 'na'출력. 근데 월단위로 뽑는데 어제 랭킹이 의미 있을까요?.?
        try:
            ranking_yest = j.find("span", class_="num")
            ranking_yest_list.append(ranking_yest.get_text(strip=True))
        except:
            ranking_yest_list.append("na")
        #가사
        lyric = soup.find(id = "d_video_summary")
        lyric_list.append(lyric.get_text(strip=True))

        
    print(
        title_list,
        artist_list,
        album_list,
        release_list,
        genre_list,
        flac_list,
        like_list,
        review_list,
        ranking_yest_list,
        lyric_list
    )

    stopper += 1
    #if문에 들어간 숫자만큼의 곡만 크롤링
    if stopper == 1:
        break

['VVS (Feat. JUSTHIS) (Prod. GroovyRoom)'] ['미란이 (Mirani),먼치맨(MUNCHMAN),Khundi Panda,머쉬베놈 (MUSHVENOM)'] ['곡명오래된 노래'] ['2020.11.21'] ['랩/힙합'] ['Flac 16/24bit'] ['총건수0'] ['0개'] ['46'] ["보여줘야겠어내가 망할 거랬던 놈들에게도내가 잘될 거라 했던 너에게도다 할게 최선갈아끼워 다 새 거My life's shining like a VVS VVS내 삶은 빛나 VVS VVS보란 듯 나와 TV에Now you see me babe내 삶은 빛나 VVS VVSSkrt-skrtHey new water new vv난 알바 째고 무대 위Yeah go get it go get it가사 위 가난이 빛나지안 가 무한리필살아봐야겠어 내 빌딩 Yeah개 같던 세상의뒤통수 치러 왔지더 크게 Callin’ ma name모두 날 보고 놀래미란이가 TV에떼버려 Tag사 새롭게 Yeah yeah타고 비행Skrr skrr 난 올라가Skrr skrr 난 빛이 나내가 뭐라 했어 Mom꺼내겠다고 포차맨 밑바닥의 소녀엄마의 술병이 날 만들어허기져 이를 꽉 물어Chit chat bout me 덤벼 겁쟁이 너VVS on ma neck 꿈 앞에녹슨 팔찌 버려 문 앞에구제 벨트 아직 허리에원망하던 과거와 춤출래인디고 카니발 Skrt-skrt목숨은 다 꺼내놓고자판기 밑 먼지 덕지덕지 붙은동전까지 털어-털어여기까지 왔지What you know about me내 속 이야기 털어놓으려지갑 털어 차린 술상 위넌 폰만 바라봤잖아이제 그 스크린에서 날 봐라 I'ma먹고서 Fly 너가 나갈 건 삔또지악플들이 돈과 함께 쏟아지니Got a nice ring to it꼬우면 네 오빠인지 아저씨인지한테일러바치고 한 판 더 뜨자 해 빛이나는 Solo라니까 뭘 그렇게 재니다 드루와 내 패기170에 60kg도 안 되지만 국보 1호 MCThe Q's certified new kingin the rap내